# Lab02 - Modelo Vetorial

### Construção do índice invertido

O método a seguir permite construir um indice invertido, já inserindo informações que serão utilizadas na construção dos modelos vetoriais, e removendo acentuação das palavras e caracteres especiais.

In [1]:
import nltk as n
import re
import math
import ast
from unicodedata import normalize
from collections import Counter

def build(data):
    inverted_index = {}
    data["noticia"] = data.titulo + " " + data.subTitulo + " " + data.conteudo
    for index, doc in data.iterrows():
        doc.noticia = doc.noticia.lower()
        words = [clear_text(word) for word in n.word_tokenize(doc.noticia)]
        freq_words = Counter(words)
        for word in words:
            inverted_index.setdefault(word, []).append((doc.idNoticia, freq_words[word]))

    for word in inverted_index.keys():
        inverted_index[word] = set(inverted_index[word])

    inverted_index_aux = {}
    M = len(data.noticia)
    for word in inverted_index.keys():
        k = len(list(inverted_index[word]))
        v_idf = idf(M, k)
        for i in range(len(list(inverted_index[word]))):
            inverted_index_aux.setdefault(word, []).append({
                "docId": list(inverted_index[word])[i][0],
                "tf": list(inverted_index[word])[i][1] ,
                "idf": v_idf
            })

    return inverted_index_aux

Este método permite calcular o *idf* de um termo e assim penalizar termos populares na nossa coleção de documentos. 

In [2]:
def idf(M, k): return  math.log((M + 1) / k)

Este método remove todos os caracteres especiais do texto bem como sua acentuação.

In [3]:
def clear_text(text):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', text)

Como as listas do documento de validaçao vem em formato de string, foi necessario a criação desse método que  converte uma string que tem o formato de uma lista em um objeto do tipo lista do python.

In [4]:
def convert_str_in_lst(lst):
    return ast.literal_eval(lst)

## Modelos Vetoriais

### Modelo binário

texto

In [5]:
def binary_representation(inverted_index, term, k = 5):
    term_search = term.split()
    result = []
    for word_term in term_search:
        if inverted_index.has_key(word_term):
            docId_word_term = []
            for i in range(len(inverted_index[word_term])):
                docId_word_term.append(inverted_index[word_term][i]["docId"])
            result.append(docId_word_term)

    i = set(result[0])
    for x in result[1:]:
        i = i & set(x)

    return list(i)[:k]

### Modelo TF

texto

In [6]:
def TF(inverted_index, term, k = 5):
    term_search = term.split()
    result_id = []
    result = {}
    for word_term in term_search:
        if inverted_index.has_key(word_term):
            docId_word_term = []
            for i in range(len(inverted_index[word_term])):
                docId_word_term.append(inverted_index[word_term][i]["docId"])
                result.setdefault(inverted_index[word_term][i]["docId"], []).append(inverted_index[word_term][i]["tf"])
            result_id.append(docId_word_term)

    i = set(result_id[0])
    for x in result_id[1:]:
        i = i & set(x)

    z = {}
    for x in i:
        z[x] = sum(result[x])
    return sorted(z, key = z.get, reverse = True)[:k]

### Modelo TF-IDF

texto

In [7]:
#coding: utf-8
def TF_IDF(inverted_index, term, k = 5):
    term_search = term.split()
    result_id = []
    result = {}
    for word_term in term_search:
        if inverted_index.has_key(word_term):
            docId_word_term = []
            for i in range(len(inverted_index[word_term])):
                docId_word_term.append(inverted_index[word_term][i]["docId"])
                result.setdefault(inverted_index[word_term][i]["docId"], []).append(inverted_index[word_term][i]["tf"] * inverted_index[word_term][i]["idf"])
            result_id.append(docId_word_term)

    i = set(result_id[0])
    for x in result_id[1:]:
        i = i & set(x)

    z = {}
    for x in i:
        z[x] = sum(result[x])
    return sorted(z, key=z.get, reverse=True)[:k]

### Modelo BM25

texto

In [27]:
def BM25(inverted_index, term, k = 5):
    term_search = term.split()
    result_id = []
    result = {}
    for word_term in term_search:
        if inverted_index.has_key(word_term):
            docId_word_term = []
            for i in range(len(inverted_index[word_term])):
                docId_word_term.append(inverted_index[word_term][i]["docId"])
                result.setdefault(inverted_index[word_term][i]["docId"], []).append(calc_BM25(inverted_index[word_term][i]["tf"]))
            result_id.append(docId_word_term)

    i = set(result_id[0])
    for x in result_id[1:]:
        i = i & set(x)

    z = {}
    for x in i:
        z[x] = sum(result[x])

    return sorted(z, key=z.get, reverse=True)[:k]

Esse método calcula o BM25 de um termo, baseado no seu *term frequency*.

In [28]:
def calc_BM25(tf, k = 5):
    return (( k + 1) * tf) / ( tf + k)

# Validação

Esse método verifica a precisão dos resultados retornados pelos algoritmos de busca baseado em modelo vetorial implementados nesse laboratório.

In [31]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

# Importação do data-set

Nesse trecho de código importamos o data-set usado em nossos algoritmos de busca, bem como importamos os dados de validação de nossos algoritmos.

In [28]:
import pandas as p
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')

data = p.read_csv("data-set/estadao_noticias_eleicao.csv", encoding = "utf-8")
data = data.replace(np.NAN, "")

validation_data = p.read_csv("validation/gabarito.csv", encoding= "utf-8")


ImportError: No module named pandas

## Chamada ao método de contrução de indice invertido

In [26]:
inverted_index_values =  build(data)

NameError: name 'data' is not defined

Como as listas contidas no data-set de validação estão no formato de string e não de lista, foi necessário fazer a conversão de string em lista.

In [ ]:

validation_data.google = validation_data.google.apply(convert_str_in_lst)
validation_data.busca_binaria = validation_data.busca_binaria.apply(convert_str_in_lst)
validation_data.tf = validation_data.tf.apply(convert_str_in_lst)
validation_data.tfidf = validation_data.tfidf.apply(convert_str_in_lst)
validation_data.bm25 = validation_data.bm25.apply(convert_str_in_lst)

Esse trecho de codigo mostra a precisão dos modelos vetoriais criados com relação ao google.

In [ ]:
binary_search = [b.binary_representation(inverted_index_values, inverted_index.clear_text(term)) for term in validation_data.str_busca]
print "Precisão gabarito busca binária: %.3f" % (valid.mapk(validation_data.busca_binaria, binary_search, k=5))
print "Precisão gabarito busca google: %.3f" % (valid.mapk(validation_data.google, binary_search, k=5))

tf_search = [tf.TF(inverted_index_values, inverted_index.clear_text(term)) for term in validation_data.str_busca]
print "Precisão gabarito busca TF: %.3f" %(valid.mapk(validation_data.tf, tf_search, k=5))
print "Precisão gabarito busca google: %.3f" %(valid.mapk(validation_data.google, tf_search, k=5))

tfidf_search = [tfidf.TF_IDF(inverted_index_values, inverted_index.clear_text(term)) for term in validation_data.str_busca]
print "Precisão gabarito busca TF-IDF: %.3f" %(valid.mapk(validation_data.tfidf, tfidf_search, k=5))
print "Precisão gabarito busca google: %.3f" %(valid.mapk(validation_data.google, tfidf_search, k=5))

bm25_search = [bm25.BM25(inverted_index_values, inverted_index.clear_text(term)) for term in validation_data.str_busca]
print "Precisão gabarito busca  BM25: %.3f" %(valid.mapk(validation_data.bm25, bm25_search, k=5))
print "Precisão gabarito busca google: %.3f" %(valid.mapk(validation_data.google, bm25_search, k=5))